## 0. Preparation

In [1]:
import turicreate

In [3]:
song_data = turicreate.SFrame('./data/song_data.sframe/')
song_data

user_id,song_id,listen_count,title,artist
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SOAKIMP12A8C130995,1,The Cove,Jack Johnson
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SOBBMDR12A8C13253B,2,Entre Dos Aguas,Paco De Lucia
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SOBXHDL12A81C204C0,1,Stronger,Kanye West
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SOBYHAJ12A6701BF1D,1,Constellations,Jack Johnson
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SODACBL12A8C13C273,1,Learn To Fly,Foo Fighters
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SODDNQT12A6D4F5F7E,5,Apuesta Por El Rock 'N'Roll ...,Héroes del Silencio
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SODXRTY12AB0180F3B,1,Paper Gangsta,Lady GaGa
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SOFGUAY12AB017B0A8,1,Stacked Actors,Foo Fighters
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SOFRQTD12A81C233C0,1,Sehr kosmisch,Harmonia
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SOHQWYZ12A6D4FA701,1,Heaven's gonna burn youreyes ...,Thievery Corporationfeat. Emiliana Torrini ...


In [15]:
## personalized model for part 3.
train_data, test_data = song_data.random_split(.8, seed=0)

personalized_model = turicreate.item_similarity_recommender.create(train_data, 
                                                                   user_id = 'user_id',
                                                                   item_id = 'song')

Warning: Ignoring columns song_id, listen_count, title, artist;

To use one of these as a rating column, specify the column name to be used as target

and use a method that allows the use of a target.

Preparing data set.

Data has 893580 observations with 66085 users and 9952 items.

...

| 372.184ms                           | 100              | 9952            |

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 1.41039s

## 1. Counting unique users:  

The method `.unique()` can be used to select the unique elements in a column of data. In this question, you will compute the number of unique users who have listened to songs by various artists.  For example, to find out the number of unique users who listened to songs by 'Kanye West', all you need to do is select the rows of the song data where the artist is 'Kanye West', and then count the number of unique entries in the ‘user_id’ column.  

Compute the number of unique users for each of these artists:  'Kanye West', 'Foo Fighters', 'Taylor Swift' and 'Lady GaGa'.  
*Save these results to answer the quiz at the end.*

In [7]:
uniq_users_artists = {artist: len(song_data[song_data['artist'] == artist].unique()) 
                      for artist in ['Kanye West', 'Foo Fighters', 'Taylor Swift', 'Lady GaGa']}
uniq_users_artists

{'Kanye West': 3775,
 'Foo Fighters': 3429,
 'Taylor Swift': 6227,
 'Lady GaGa': 4129}

### Q1. Which of the artists below have had the most unique users listening to their songs?

  - [ ] Kanye West
  - [ ] Foo Fighters
  - [x] Taylor Swift
  - [ ] Lady GaGa

## 2. Using groupby-aggregate to find the most popular and least popular artist

Each row of song_data contains the number of times a user listened to particular song by a particular artist.  If we would like to know how many times any song by 'Kanye West' was listened to, we need to select all the rows where `‘artist’=='Kanye West'` and sum the ‘listen_count’ column.  If we would like to find the most popular artist, we would need to follow this procedure for each artist, which would be very slow.  Instead, you will learn about a very important method: `groupby()`.

The documentation about [groupby here](https://apple.github.io/turicreate/docs/api/generated/turicreate.SFrame.groupby.html).  The `.groupby` method computes an aggregate (in our case, the sum of the ‘listen_count’) for each distinct value in a column (in our case, the ‘artist’ column).  

Follow these steps to find the most popular artist in the dataset: 

The .groupby method has two important parameters:

  1. `key_column_names`, which takes the column we want to group, in our case, ‘artist’
  1. `operations`, where we define the aggregation operation we using, in our case, we want to sum over the ‘listen_count’.  


In [12]:
import turicreate.aggregate as agg

popular_artists = song_data.groupby(key_column_names='artist', 
                                    operations={'total_count': agg.SUM('listen_count')}).sort('total_count', ascending=False)

popular_artists.head(10)

artist,total_count
Kings Of Leon,43218
Dwight Yoakam,40619
Björk,38889
Coldplay,35362
Florence + The Machine,33387
Justin Bieber,29715
Alliance Ethnik,26689
OneRepublic,25754
Train,25402
The Black Keys,22184


In [13]:
popular_artists.tail(1)

artist,total_count
William Tabbert,14


### Q2. Which of the artists below is the most popular artist, the one with highest total listen_count, in the data set?

  - [ ] Taylor Swift
  - [x] Kings of Leon
  - [ ] Coldplay
  - [ ] Lady GaGa
  
### Q3.  Which of the artists below is the least popular artist, the one with smallest total listen_count, in the data set?

  - [x] William Tabbert
  - [ ] Velvet Underground & Nico
  - [ ] Kanye West
  - [ ] The Cool Kids

## 3. [Optional] Using groupby-aggregate to find the most recommended songs
Now that we learned how to use `.groupby()` to compute aggregates for each value in a column, let’s use it to find the song that is *most recommended by the personalized_model* model we learned in the Jupyter notebook above. Follow these steps to find the most recommended song:

  - Split the data into 80% training, 20% testing, using seed=0, as was done in the Jupyter notebook for this week.
  - Train an `item_similarity_recommender`, as done in the Jupyter notebook, using the training data (called it `personalized_model`).
  - Next, we are going to make recommendations for the users in the test data, but  there are over 200,000 users (58,628 unique users) in the test set.  
    Computing recommendations for these many users can be slow in some computers.  Thus, we will use only the first ~10,000~ 50,000 users only in this question.  
    Using this command to select this subset of users:
    ```python
    LIMIT = 50_000
    subset_test_users = test_data['user_id'].unique()[0:LIMIT]
    ```
    
  - Then we compute one recommended song for each of these test users, using:
    ```python
    personalized_model.recommend(subset_test_users, k=1)
    ```
    
  - Finally, we can use `.groupby()` to find the most recommended song. For this we will use the `COUNT` aggregator. We will then sort the result

In [17]:
## data already split (cf. prep.)
## personalized_model (cf. prep.)

LIMIT = 50_000
subset_test_users = test_data['user_id'].unique()[0:LIMIT]

one_recom_song = personalized_model.recommend(subset_test_users, k=1)

most_recom_song = one_recom_song.groupby(key_column_names=['song'], 
                                         operations={'count': agg.COUNT()}).sort('count', ascending=False)

most_recom_song.head(10)  ## most recomended: Undo - Björk

recommendations finished on 1000/50000 queries. users per second: 63363.3

recommendations finished on 2000/50000 queries. users per second: 67916.3

recommendations finished on 3000/50000 queries. users per second: 71133.9

recommendations finished on 4000/50000 queries. users per second: 70631.4

recommendations finished on 5000/50000 queries. users per second: 70130.2

...

recommendations finished on 47000/50000 queries. users per second: 62894.8

recommendations finished on 48000/50000 queries. users per second: 61549

recommendations finished on 49000/50000 queries. users per second: 60201.7

recommendations finished on 50000/50000 queries. users per second: 56141.5

song,count
Undo - Björk,2186
Secrets - OneRepublic,1910
Revelry - Kings Of Leon,1106
You're The One - DwightYoakam ...,810
Sehr kosmisch - Harmonia,559
Fireflies - CharttraxxKaraoke ...,551
Hey_ Soul Sister - Train,542
Horn Concerto No. 4 in Eflat K495: II. Romance ...,431
OMG - Usher featuringwill.i.am ...,337
The Scientist - Coldplay,225
